In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import train,prepare,evaluate,tune
from models.Encoders import NRMS_Encoder, FIM_Encoder, NPA_Encoder, Pipeline_Encoder, MHA_Encoder, Bert_Encoder, RNN_Encoder, CNN_Encoder
from models.Interactors import FIM_Interactor, KNRM_Interactor
from models.SFI import SFI_gating, SFI_gating_MultiView

In [2]:
name='sfi'
hparams = {
    'scale':'demo',
    'mode':'tune',
    'device':'cpu',
    'epochs':5,
    'val_freq':1,
    'batch_size':3,
    'his_size':50,
    'k':50,
    'threshold':0.5,
    'contra_num':0,
    'title_size':20,
    'abs_size':40,
    'npratio':4,
    'dropout_p':0.2,
    'embedding_dim':300,
    'value_dim':16,
    'query_dim':200,
    'head_num':16,
    'filter_num':150,
    'select':'gating',
    'level':1,
    'metrics':'auc,mean_mrr,ndcg@5,ndcg@10',
    # 'attrs': ['abs','vert','subvert'],
    'attrs': ['title'],
    'save_step':[0],
    'spadam':True,
    'interval':10,
}
# torch.cuda.set_device(hparams['device'])

In [7]:
# hparams['validate'] = True
vocab, loaders = prepare(hparams, pin_memory=False)

[2021-04-16 16:01:51,839] INFO (root) Hyper Parameters are
{'scale': 'demo', 'mode': 'tune', 'device': 'cpu', 'epochs': 5, 'val_freq': 1, 'batch_size': 3, 'his_size': 50, 'k': 50, 'threshold': 0.5, 'contra_num': 0, 'title_size': 20, 'abs_size': 40, 'npratio': 4, 'dropout_p': 0.2, 'embedding_dim': 300, 'value_dim': 16, 'query_dim': 200, 'head_num': 16, 'filter_num': 150, 'select': 'gating', 'level': 1, 'metrics': 'auc,mean_mrr,ndcg@5,ndcg@10', 'attrs': ['title'], 'save_step': [0], 'spadam': True, 'interval': 10, 'name': 'sfi-fim-encoder-gating'}
[2021-04-16 16:01:51,841] INFO (root) preparing dataset...
[2021-04-16 16:01:56,383] INFO (torchtext.vocab) Loading vectors from .vector_cache/glove.840B.300d.txt.pt


In [8]:
record = next(iter(loaders[0]))

In [16]:
encoder = FIM_Encoder(hparams, vocab)
interactor = FIM_Interactor()
# interactor = KNRM_Interactor()

hparams['name'] = '-'.join([name,encoder.name,hparams['select']])

# sfi = SFI_gating_MultiView(hparams, encoder, interactor).to(hparams['device'])
sfi = SFI_gating(hparams, encoder, interactor).to(hparams['device'])

sfi.load_state_dict(torch.load('data/model_params/sfi-fim-fim-gating/large_epoch2_step16916_[hs=50,topk=50,contra=0,attrs=title].model', map_location=hparams['device'])['model'])

<All keys matched successfully>

In [20]:
record['clicked_tit']

tensor([[[False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False],
         [False]],

        [[False],
         [False],
         [False],
         [False],
         [False],
        

In [17]:
sfi(record)

tensor([0.8245, 0.7307, 0.8135, 0.7637, 0.7833, 0.7387, 0.9513, 0.3759, 0.3761,
        0.3755, 0.3765, 0.3791, 0.3763, 0.3771, 0.3763, 0.3794, 0.3749, 0.3760,
        0.3742, 0.3761, 0.3776, 0.3762, 0.3766, 0.3785, 0.3742, 0.3750, 0.3760,
        0.3751, 0.3757, 0.3772, 0.3746, 0.3758, 0.3765, 0.3789, 0.3744, 0.3754,
        0.3770, 0.3768, 0.3747, 0.3739, 0.3758, 0.3788, 0.3756, 0.3745, 0.3744,
        0.3764, 0.3756, 0.3753, 0.3744, 0.3743], grad_fn=<SelectBackward>) tensor([[[[3.1615e-01, 4.1319e-01, 5.2854e-01,  ..., 0.0000e+00,
           5.6235e-01, 1.5901e-01],
          [0.0000e+00, 5.1890e-01, 1.9741e-01,  ..., 0.0000e+00,
           1.0225e+00, 0.0000e+00],
          [0.0000e+00, 1.5739e-01, 1.1076e+00,  ..., 1.5680e+00,
           6.1721e-02, 0.0000e+00]],

         [[0.0000e+00, 5.5570e-01, 4.7069e-01,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 3.6039e-01,  ..., 0.0000e+00,
           6.1572e-01, 0.0000e+00],
          [0.0000e

tensor([[-0.3942, -2.9030, -2.0372, -2.6688, -2.6424],
        [-0.3422, -1.7006, -3.0423, -3.1470, -4.1044],
        [-1.6755, -2.2884, -2.7162, -1.4525, -0.8886]],
       grad_fn=<LogSoftmaxBackward>)

In [ ]:
sfi = tune(sfi, hparams, loaders)

In [ ]:
evaluate(sfi, hparams, loaders[1], loading=True)

In [ ]:
hparams['name'] = '-'.join([name,'nrms',hparams['select']])
nrms_encoder = NRMS_Encoder(hparams, vocab)
sfi = SFI_gating(hparams, nrms_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['name'] = '-'.join([name,'npa',hparams['select']])

hparams['user_dim'] = 200
hparams['query_dim'] = 200
hparams['filter_num'] = 400

npa_encoder = NPA_Encoder(hparams, vocab, len(loaders[0].dataset.uid2index))
sfi = SFI_gating(hparams, npa_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['name'] = '-'.join([name,'pipeline',hparams['select']])

hparams['pipeline'] = 'sfi-fim'
# hparams['scale'] = 'large'

ppl_encoder = Pipeline_Encoder(hparams)
sfi = SFI_gating(hparams, ppl_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['name'] = '-'.join([name,'mha',hparams['select']])

mha_encoder = MHA_Encoder(hparams, vocab)
sfi = SFI_gating(hparams, mha_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['name'] = '-'.join([name,'lstm',hparams['select']])
hparams['hidden_dim'] = 256

rnn_encoder = RNN_Encoder(hparams,vocab)
sfi = SFI_gating(hparams, rnn_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['name'] = '-'.join([name,'bert',hparams['select']])

hparams['level'] = 2
# hparams['batch_size'] = 10
# hparams['bert'] = 'bert-base-uncased'
hparams['bert'] = 'albert-base-v2'

_, loaders = prepare(hparams)

In [ ]:
bert_encoder = Bert_Encoder(hparams)
sfi = SFI_gating(hparams, bert_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

## The rest is for development